The following code was used to evaluate the results generated by RV EXCALIBER. Here we validate the burden matrix has the correct variants for each patient. Along with validating allele counts generated by RV EXCALIBER

# STEP 1 

Step 1 loads gene region information from the ANNOVAR file.
Each line in the file contains a gene name along with its genomic location (chromosome, start, and stop positions).
These regions are stored in a dictionary called gene_regions, where:

The key is the gene name.

The value is a list of one or more tuples, each representing a (chromosome, start, stop) for that gene. 

In [2]:
import pandas as pd

# File Paths
base_path = "/project/pi_rachel_melamed_uml_edu/neuroblastoma/maddie/double_checking/"
vcf_file = base_path + "output_hg19.vcf"
burden_matrix_file = base_path + "final_output_RVBurdenMatrix_0.9_0_nfe_rcc.txt"  
annovar_file = base_path + "hg19_refGene.txt"  

print("Files successfully loaded", flush=True)

# Step 1: Load Gene Regions from ANNOVAR
print("Loading ANNOVAR file", flush=True)
gene_regions = {}

with open(annovar_file, 'r') as annovar:
    for line in annovar:
        parts = line.strip().split("\t")
        if len(parts) < 5:  # Skip malformed lines
            continue
        gene_name = parts[12]  # Gene name
        chrom = parts[2]       # Chromosome
        start = int(parts[4])  # Start position
        stop = int(parts[5])   # Stop position

        if gene_name not in gene_regions:
            gene_regions[gene_name] = []  # Initialize list for regions
        gene_regions[gene_name].append((chrom, start, stop)) # Add chrom start and stop to list

print(f"Loaded {len(gene_regions)} genes from ANNOVAR", flush=True)


Files successfully loaded
Loading ANNOVAR file


FileNotFoundError: [Errno 2] No such file or directory: '/project/pi_rachel_melamed_uml_edu/neuroblastoma/maddie/double_checking/hg19_refGene.txt'

# Step 2 

Step 2 loads the variants from the VCF file.
Each patient's variants are saved in a dictionary called patient_variants, where:

The keys are patient IDs.

The values are lists of tuples containing the chromosome and position of each variant that the patient carries. Only variants where the patient has at least one alternate allele (e.g., "0/1", "1/1", or "1/2") are stored.

In [ ]:
# Step 2: Load Variants from VCF (ONLY Store Variants)
print("Loading VCF file", flush=True)
patient_variants = {}  # {patient: [(chrom, pos), ...]}
patient_ids = []

with open(vcf_file, 'r') as vcf:
    for line in vcf:
        if line.startswith("#CHROM"):
            header_parts = line.strip().split("\t")
            patient_ids = header_parts[9:]  # Extract patient IDs
            for patient_id in patient_ids:
                patient_variants[patient_id] = [] # Set patient ids as keys
            continue

        if line.startswith("#"):
            continue  # Skip header lines

        parts = line.strip().split("\t")
        chrom = parts[0]
        pos = int(parts[1])
        samples = parts[9:]

        for idx, genotype in enumerate(samples): # Loops through each patients genotype
            patient_id = patient_ids[idx] # Uses idx to get right patient id
            genotype_value = genotype.split(":")[0]  # Extract only genotype (GT)

            if "1" in genotype_value or "2" in genotype_value:
                patient_variants[patient_id].append((chrom, pos))  # Store variant location

print(f"Loaded variants for {len(patient_variants)} patients from VCF", flush=True)


# STEP 3

Step 3 loads in the burden matrix using pandas

In [ ]:
# Step 3: Load Burden Matrix
print("Loading Burden Matrix", flush=True)
burden_matrix = pd.read_csv(burden_matrix_file, sep=r"\s+", index_col=0, engine="python")

print(f"Loaded burden matrix with {burden_matrix.shape[0]} genes and {burden_matrix.shape[1]} patients", flush=True)

# STEP 4

Step 4 validates the burden matrix by checking whether each reported variant for a gene is actually supported by the VCF data.

First, it checks whether any patients listed in the burden matrix are missing from the VCF. If so, it prints a warning and skips those patients.

Then, for each gene in the burden matrix, it:

Cleans the gene name (in case it's a fusion gene, taking only the first part before a semicolon).

Checks whether the burden matrix says the patient has a variant (1).

If so, it checks whether the cleaned gene name exists in the ANNOVAR gene regions.

If it does, it loops through each region for that gene and compares all of that patient's variant locations.

If any of the patient’s variants fall within the gene’s start/stop region, it's considered a valid match.

If a match is found, it’s added to the matches list. If not, it’s logged in the errors list as a mismatch between the burden matrix and VCF.

In [ ]:
# Step 4: Validate Burden Matrix Variants Using VCF
errors = []
matches = []

print("Checking for mismatches between Burden Matrix and VCF...", flush=True)

for patient in burden_matrix.columns:
    if patient not in patient_variants:  # If patient is missing in VCF, flag it
        print(f"Warning: Patient {patient} is in Burden Matrix but missing from VCF!")
        continue

    for gene in burden_matrix.index:
        cleaned_gene = gene.split(";")[0]  # Take the first gene name
        reported_in_matrix = burden_matrix.loc[gene, patient]

        # Only check genes marked as '1' (has a variant in burden matrix)
        if reported_in_matrix == 1:
            has_variant = False

            # Check if the patient has a variant in the gene region
            if cleaned_gene in gene_regions:
                for chrom, start, stop in gene_regions[cleaned_gene]:
                    for v_chrom, v_pos in patient_variants[patient]:
                        if v_chrom == chrom and start <= v_pos <= stop:
                            has_variant = True
                            break
                    if has_variant:
                        break

            # Log results
            if has_variant:
                matches.append(f"Match: {patient} has a variant in {gene} (VCF & Burden Matrix)")
            else:
                errors.append(f"Mismatch: {patient} is marked as 1 in Burden Matrix but has no variant in VCF for {gene}")

print("Variant check completed.", flush=True)


# STEP 5

Output matches and mismatches

In [ ]:
# Step 5: Print Matches & Errors
if matches:
    print(f"Found {len(matches)} correct matches! Showing first 10:")
    for match in matches[:10]:
        print(match)

if errors:
    print(f"Found {len(errors)} mismatches! Showing first 10:")
    for error in errors[:10]:
        print(error)
else:
    print("No mismatches found! Burden Matrix is consistent with VCF.")

print("Updated variant checking job completed.", flush=True)

# Checking allele counts

Checking Allele Counts

In this step, we calculate the total number of patients with a variant in each gene.

First, we sum across each row of the burden matrix using .sum(axis=1), which gives us the total number of 1s per gene (i.e., the number of patients who have a variant in that gene).

Then, we convert the resulting Series into a DataFrame with two columns: "Gene" and "Allele_Count".

Finally, we save the allele counts to a tab-separated text file for downstream analysis.

In [ ]:
import pandas as pd

print("Loading Burden Matrix...", flush=True)

# File Path to the Burden Matrix
burden_matrix_file = "/project/pi_rachel_melamed_uml_edu/neuroblastoma/maddie/double_checking/final_output_RVBurdenMatrix_0.9_0_nfe_rcc.txt"

# Load Burden Matrix
burden_matrix = pd.read_csv(burden_matrix_file, sep=r"\s+", index_col=0, engine="python")

print(f"Loaded burden matrix with {burden_matrix.shape[0]} genes and {burden_matrix.shape[1]} patients", flush=True)

# Compute allele count per gene (sum across patient columns)
allele_counts = burden_matrix.sum(axis=1)  

# Convert to DataFrame
allele_counts_df = allele_counts.reset_index()
allele_counts_df.columns = ["Gene", "Allele_Count"]

# Save to output file
output_file = "/project/pi_rachel_melamed_uml_edu/neuroblastoma/maddie/count_alleles/burden_matrix_allele_counts.txt"
allele_counts_df.to_csv(output_file, sep="\t", index=False)

# Print first 10 results
print("Allele counts computed! First 10 genes:")
print(allele_counts_df.head(10))

print(f"Allele counts saved to: {output_file}")